In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
df = pd.read_csv("sales_pos.csv")
df.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


### Q1.

In [3]:
df_q1 = df[["prod", "job", "purchase"]].copy()
df_q1.head(2)

,prod,job,purchase
0,P00069042,10,8370
1,P00248942,10,15200


In [5]:
df_q1.groupby("prod")["purchase"].sum().max()

27995166

In [ ]:
ser_q1_agg = df_q1.groupby("prod")["purchase"].sum()
ser_q1_agg[ser_q1_agg == ser_q1_agg.max()]

In [ ]:
best_prod = df_q1.groupby("prod")["purchase"].sum().idxmax()
best_prod

In [10]:
df_q1_sub = df_q1.loc[df_q1["prod"] == best_prod, ]
df_q1_sub.head(2)

,prod,job,purchase
667,P00025442,17,19706
749,P00025442,7,15212


In [12]:
df_q1_sub["job"].value_counts().idxmax()

4

### Q2.

In [ ]:
df_u1 = df.loc[df["user"] == 1, ["prod_cat1", "prod_cat2", "prod_cat3"]]
df_u1 = df_u1.reset_index(drop = True)
df_u1 = df_u1.fillna(0)
df_u1.head(3)

In [ ]:
len(df_u1)

In [20]:
df_u1.sort_values(["prod_cat1", "prod_cat2", "prod_cat3"])

In [26]:
df_u1.drop_duplicates().shape

In [24]:
df_u1["prod_cat1"] = df_u1["prod_cat1"].astype("int").astype("str")
df_u1["prod_cat2"] = df_u1["prod_cat2"].astype("int").astype("str")
df_u1["prod_cat3"] = df_u1["prod_cat3"].astype("int").astype("str")
df_u1["prod_cat" ] = df_u1["prod_cat1"] + "-" + df_u1["prod_cat2"] + "-" + df_u1["prod_cat3"]

In [25]:
df_u1.head(3)

,prod_cat1,prod_cat2,prod_cat3,prod_cat
0,3,0,0,3-0-0
1,1,6,14,1-6-14
2,12,0,0,12-0-0


In [27]:
df_u1["prod_cat"].nunique()

21

In [28]:
df["age_group"].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [32]:
df_q2 = df.loc[df["age_group"] == "26-35", ]
df_q2 = df_q2[["user", "marital", "prod_cat1", "prod_cat2", "prod_cat3"]]
df_q2 = df_q2.reset_index(drop = True)
df_q2 = df_q2.fillna(0)

In [34]:
df_q2["prod_cat1"] = df_q2["prod_cat1"].astype("int").astype("str")
df_q2["prod_cat2"] = df_q2["prod_cat2"].astype("int").astype("str")
df_q2["prod_cat3"] = df_q2["prod_cat3"].astype("int").astype("str")
df_q2["prod_cat" ] = df_q2["prod_cat1"] + "-" + df_q2["prod_cat2"] + "-" + df_q2["prod_cat3"]

In [35]:
df_q2.head(2)

,user,marital,prod_cat1,prod_cat2,prod_cat3,prod_cat
0,3,0,1,2,0,1-2-0
1,5,1,8,0,0,8-0-0


In [37]:
df_q2_agg = df_q2.groupby(["user", "marital"])["prod_cat"].nunique().reset_index()
df_q2_agg.head(2)

,user,marital,prod_cat
0,3,0,18
1,5,1,43


In [38]:
stat_m0 = df_q2_agg.loc[df_q2_agg["marital"] == 0, "prod_cat"].mean()
stat_m1 = df_q2_agg.loc[df_q2_agg["marital"] == 1, "prod_cat"].mean()
round(abs(stat_m0 - stat_m1), 2)

0.13

### Q3.

In [39]:
df_q3_user = df[["user", "gender", "age_group", "job", "city", "marital"]].copy()
df_q3_user.head(2)

,user,gender,age_group,job,city,marital
0,1,F,0-17,10,A,0
1,1,F,0-17,10,A,0


In [46]:
df_q3_user["user"].nunique()

In [47]:
df_q3_user = df_q3_user.drop_duplicates().reset_index(drop = True)
df_q3_user.shape

In [44]:
df_q3_agg = df.groupby("user")[["prod", "purchase"]].agg({"prod": "nunique",
                                                          "purchase": "sum"})
df_q3_agg = df_q3_agg.reset_index()
df_q3_agg.head(2)

,user,prod,purchase
0,1,35,334093
1,2,77,810472


In [45]:
df_q3 = df_q3_user.merge(df_q3_agg, on = "user")
df_q3.head(3)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,F,0-17,10,A,0,35,334093
1,2,M,55+,16,C,0,77,810472
2,3,M,26-35,15,A,0,29,341635


In [50]:
cols = ["user", "gender", "age_group", "job", "city", "marital"]
df_q3 = df.groupby(cols)[["prod", "purchase"]].agg({"prod": "nunique",
                                                    "purchase": "sum"})
df_q3 = df_q3.reset_index()
df_q3.head(3)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,F,0-17,10,A,0,35,334093
1,2,M,55+,16,C,0,77,810472
2,3,M,26-35,15,A,0,29,341635


In [51]:
df_q3["gender"] = df_q3["gender"].replace({"M": 1, "F": 0})

In [54]:
arr_u = df_q3["age_group"].unique()
# arr_u = arr_u.sort() # 안됨!!
arr_u.sort() # .... ☹
arr_u

array(['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+'],
      dtype=object)

In [56]:
ser_repl = pd.Series(range(7), index = arr_u)
ser_repl

In [57]:
pd.crosstab(df_q3["age_group"],
            df_q3["age_group"].replace(ser_repl))

age_group,0,1,2,3,4,5,6
age_group,,,,,,,
0-17,218,0,0,0,0,0,0
18-25,0,1069,0,0,0,0,0
26-35,0,0,2053,0,0,0,0
36-45,0,0,0,1167,0,0,0
46-50,0,0,0,0,531,0,0
51-55,0,0,0,0,0,481,0
55+,0,0,0,0,0,0,372


In [58]:
df_q3["age_group"] = df_q3["age_group"].replace(ser_repl)

In [59]:
df_q3.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,0,0,10,A,0,35,334093
1,2,1,6,16,C,0,77,810472


In [62]:
# df_q3_dum = pd.get_dummies(df_q3, dtype = "int")
df_q3_dum = pd.get_dummies(df_q3, columns = ["job", "city"], dtype = "int")
df_q3_dum = df_q3_dum.drop(columns = "user")
df_q3_dum.head(2)

,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,job_4,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,0,0,0,35,334093,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,6,0,77,810472,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [63]:
arr_q3_nor = MinMaxScaler().fit_transform(df_q3_dum)
arr_q3_nor[:1, ]

array([[0.        , 0.        , 0.        , 0.02843137, 0.02739807,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        ]])

In [ ]:
model_kmeans = KMeans(n_clusters = 7, random_state = 123)
model_kmeans.fit(arr_q3_nor)

In [68]:
round(silhouette_score(arr_q3_nor, labels = model_kmeans.labels_), 2)

0.18

In [73]:
df_q3["cluster"] = model_kmeans.labels_
df_q3.head(2)

,user,gender,age_group,job,city,marital,prod,purchase,cluter,cluster
0,1,0,0,10,A,0,35,334093,4,4
1,2,1,6,16,C,0,77,810472,5,5


In [76]:
df_q3["cluster"].value_counts()

cluster
5    1291
4    1045
0     980
2     868
6     716
1     503
3     488
Name: count, dtype: int64

In [75]:
df_q3.groupby("cluster")["age_group"].mean()